In [2]:
using Printf, CSV, DataFrames, Statistics, Glob, XLSX, Dates, Plots, TimeSeries, JSON

In [3]:
directory = joinpath(homedir(), "research", "GADS_Processed", "raw_data")
files = glob("*_state_Texas_raw.csv", directory)
x = Dict()
i = 1
mydict = Dict([("A", 1), ("RS", 1), ("NC", 1),  ("IR", 1), ("MB", 1), ("F", 2), ("U1", 2), ("U2" , 2), ("U3" , 2), ("D1", 2), ("D2", 2), ("D3", 2), ("SF", 2), ("PO", 3), ("PE", 3), ("PD", 3), ("DP", 3), ("MO", 4), ("ME", 4), ("DM", 4), ("D4", 4), ("RU", 5), ("DR", 5), ("TC", 5)])
agedict = Dict([(1, DateTime(1980, 1, 1)), (2, DateTime(1985, 1, 1)), (3, DateTime(1995, 1, 1)), (4, DateTime(2005, 1, 1)), (5, DateTime(2015, 1, 1)), (6, DateTime(2020, 1, 1))])
types_dict = Dict("EventStartDT" => DateTime, "EventEndDT" => DateTime)
rating_dict = Dict([(1, 50), (2, 150), (3, 200)]);

In [18]:
start_date_g = DateTime(2013,1,1,6)
end_date_g = DateTime(2024,1,1,6)
date_range_g = [k for k in start_date_g:Hour(1):end_date_g]

myrange_g = [k for k in 0:1:round(end_date_g - start_date_g, Hour).value]


for f in files
    @printf "%s \n" f

    mymatch = match(r"/home/emco4286/research/GADS_Processed/raw_data/gen_(\d+)_type_(\d+)_dtgrp_(\d+)_rating_(\d+)_state_Texas_raw.csv", f)
    id = parse(Int32, mymatch[1])
    type = parse(Int32, mymatch[2])
    dtgrp = parse(Int32, mymatch[3])
    rating = parse(Int32, mymatch[4])

    df = DataFrame(CSV.File(f, types=types_dict))
    df = df[typeof.(df.EventEndDT) .== DateTime, :] # for missing DateTime values
    df = df[df.EventTypeCode .!= "RS", :]
    sort!(df, :EventStartDT)
    transform!(df, :EventTypeCode => ByRow(x -> mydict[x]), :EventTypeCode)

    state = convert(Array{Union{Missing, Float64}}, fill(1, length(myrange_g))) # to allow missing values

    if size(df)[1] < 1 # Happens if event log is just full of RS events
        print("No")
    else
        # Set state to "missing" before the generator comes online
        birthdate = min(agedict[df[1, :CommercialDT_grp]], minimum(df[!, :EventStartDT]))
        if birthdate > start_date_g
            birth_hour = round(birthdate - start_date_g, Hour).value
            state[1:birth_hour] .= missing
        end

        # Hour that outage starts
        time_vector = map(x -> round(x - start_date_g, Hour), df[!, :EventStartDT])

        # Set state to zero during an event
        for k in [c for c in range(1, step=1, length=length(time_vector))] # k is an index
            start_time_string = round(df[k, :EventStartDT], Hour)
            end_time_string = round(df[k, :EventEndDT], Hour)

            cause = df[k, :EventTypeCode_function]

            if cause == 1 # RS
                continue
            end

            start_hour = time_vector[k].value # integer
            duration = round(end_time_string - start_time_string, Hour).value
            end_hour = start_hour + duration

            start_index = max(1, start_hour)

            matching_idx = findall(x -> x == end_hour, myrange_g)
            
            if length(matching_idx) < 1
                end_index = length(state)
            else
                end_index = matching_idx[1]
            end

            # If event is unit getting retired, set state as missing until the end of the period and skip to the next unit
            if cause == 5
                state[start_index:end] .= missing
            else
                state[start_index:end_index] .= 0
            end

        end

        trajectory = DataFrame(x=date_range_g, y=state)

        fname = @sprintf "gen_%d_type_%d_dtgrp_%d_rating_%d_state_Texas_raw.csv" id type dtgrp rating 
        path = joinpath(homedir(), "research", "GADS_Processed", "trajectories", fname)
        CSV.write(path, trajectory)
    end
end

/home/emco4286/research/GADS_Processed/raw_data/gen_10001_type_400_dtgrp_5_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10034_type_851_dtgrp_2_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10035_type_851_dtgrp_2_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10036_type_852_dtgrp_2_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10037_type_851_dtgrp_4_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10038_type_851_dtgrp_4_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10039_type_852_dtgrp_4_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10040_type_851_dtgrp_4_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10041_type_851_dtgrp_4_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_100

┌ Warning: thread = 1 warning: error parsing DateTime around row = 103, col = 1: "NULL,", error=INVALID: DELIMITED | INVALID_DELIMITER 
└ @ CSV /home/emco4286/.julia/packages/CSV/XLcqT/src/file.jl:595


/home/emco4286/research/GADS_Processed/raw_data/gen_10809_type_800_dtgrp_4_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10810_type_800_dtgrp_4_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10840_type_861_dtgrp_2_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10841_type_861_dtgrp_2_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10842_type_861_dtgrp_2_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10843_type_861_dtgrp_2_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10844_type_861_dtgrp_2_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10845_type_861_dtgrp_3_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_10846_type_862_dtgrp_2_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_110

┌ Warning: thread = 1 warning: error parsing DateTime around row = 42, col = 1: "NULL,", error=INVALID: DELIMITED | INVALID_DELIMITER 
└ @ CSV /home/emco4286/.julia/packages/CSV/XLcqT/src/file.jl:595


/home/emco4286/research/GADS_Processed/raw_data/gen_11826_type_852_dtgrp_5_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_11843_type_852_dtgrp_5_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_11844_type_851_dtgrp_5_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_11845_type_851_dtgrp_5_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_11900_type_100_dtgrp_5_rating_1_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_11909_type_852_dtgrp_4_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_11953_type_300_dtgrp_5_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_11954_type_300_dtgrp_5_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_11968_type_861_dtgrp_5_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_119

...

/home/emco4286/research/GADS_Processed/raw_data/gen_5058_type_200_dtgrp_3_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5059_type_200_dtgrp_3_rating_3_state_Texas_raw.csv 


┌ Warning: thread = 1 warning: error parsing DateTime around row = 4247, col = 1: "NULL,", error=INVALID: DELIMITED | INVALID_DELIMITER 
└ @ CSV /home/emco4286/.julia/packages/CSV/XLcqT/src/file.jl:595
┌ Warning: thread = 1 warning: error parsing DateTime around row = 4248, col = 1: "NULL,", error=INVALID: DELIMITED | INVALID_DELIMITER 
└ @ CSV /home/emco4286/.julia/packages/CSV/XLcqT/src/file.jl:595


/home/emco4286/research/GADS_Processed/raw_data/gen_5591_type_850_dtgrp_4_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5592_type_850_dtgrp_4_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5593_type_850_dtgrp_4_rating_3_state_Texas_raw.csv 


┌ Warning: thread = 1 warning: error parsing DateTime around row = 2100, col = 1: "NULL,", error=INVALID: DELIMITED | INVALID_DELIMITER 
└ @ CSV /home/emco4286/.julia/packages/CSV/XLcqT/src/file.jl:595


/home/emco4286/research/GADS_Processed/raw_data/gen_5594_type_850_dtgrp_4_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5807_type_100_dtgrp_1_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5808_type_100_dtgrp_1_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5809_type_100_dtgrp_1_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5810_type_100_dtgrp_1_rating_2_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5811_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5812_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5814_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5819_type_100_dtgrp_1_rating_3_state_Texas_raw.csv 
/home/emco4286/research/GADS_Processed/raw_data/gen_5821_type_10